## Importing required libraries

In [1]:
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import pickle

## Getting the dataframe which has all the diagnosis codes for all 10000 patients

In [4]:
df1 = pd.read_csv('10k-diagnosis.csv',header=None)
df1 = df1.rename(columns={0: 'patient_id', 1: 'diagnosis_coded'})
df1.head()


list_index=[]
for i in range(df1.shape[0]): #Taking only the 10 diagnoses
    if df1.iloc[i,1]=='1385187' or df1.iloc[i,1]=='1385109' or df1.iloc[i,1]=='1384870' or df1.iloc[i,1]=='1384841' or df1.iloc[i,1]=='1384995' or df1.iloc[i,1]=='1397715' or df1.iloc[i,1]=='1384886' or df1.iloc[i,1]=='1474651' or df1.iloc[i,1]=='1385083' or df1.iloc[i,1]=='1385121':
        #print(i)
        list_index.append(i)
        
df1=df1.iloc[list_index]
df1=df1.reset_index()
df1=df1.drop(['index'], axis=1)
df1=df1.drop_duplicates()
df1=df1.reset_index()
df1=df1.drop(['index'], axis=1)
df1

,patient_id,diagnosis_coded
0,19563570,1384841
1,19557987,1384841
2,19557524,1384841
3,19556594,1384841
4,19555886,1384841
...,...,...
12577,17008429,1474651
12578,17008031,1474651
12579,17006542,1474651
12580,17002218,1474651


In [ ]:
target_variable_df=pd.DataFrame()
target_variable_df["patient_id"]=df1["patient_id"].unique()
list_diagnosis=list(df1["diagnosis_coded"].unique())

list_diagnosis
for i in range(len(list_diagnosis)):
    target_variable_df[list_diagnosis[i]]=np.nan


#target_variable_df
for i in range(df1.shape[0]):
    print(i)
#     if i>1:
#         break
    patient_id=df1["patient_id"][i]
    diagnosis_id=df1["diagnosis_coded"][i]
    index_of_this_patient=int(target_variable_df.query("patient_id == @patient_id").index.values)
    #print(index_of_this_patient)
    target_variable_df[diagnosis_id][index_of_this_patient]=1

target_variable_df.head()
    

In [6]:
target_variable_df.describe()

,patient_id,1384841,1384870,1384886,1384995,1385083,1385109,1385121,1385187,1397715,1474651
count,1.000000e+04,1334.0,1462.0,1048.0,463.0,1034.0,1689.0,977.0,2334.0,1159.0,1082.0
mean,1.849807e+07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,8.663063e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.699898e+07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,1.775001e+07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,1.850531e+07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,1.924246e+07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,1.999978e+07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [7]:
# merged = pd.merge(df, df1, on ='patient_id')
# merged=merged.drop_duplicates()
# merged=merged.reset_index(drop=True)
# merged

In [8]:
# merged.to_csv("Final_merged_dataframe_only_10_diagnosis.csv")

In [9]:
# data=pd.read_csv("Final_merged_dataframe_only_10_diagnosis.csv")
# data=data.drop(['Unnamed: 0'], axis=1)
# # data=data.reset_index(inplace=True)
# data

## Getting the first dataframe which has all the patients' (n=8297) value_numeric for all the concepts

In [39]:
df = pd.read_csv('10k-obs.csv',sep="\t",header=None)
df = df.rename(columns={0:'patient_id',1: 'concept_id', 2: 'value_numeric',3:'value_coded'})
df=df.drop(['value_coded'], axis=1) #we do not need value_coded column anymore
df["value_numeric"] = df["value_numeric"].str.replace(r'\\N','np.nan', regex=True) #replacing \N with 0
df=df.replace("np.nan",np.nan)
df["value_numeric"]=df['value_numeric'].astype(float) #converting the datatypes of value_numeric as float
df=df.dropna()
df=df.reset_index(drop=True)
data=df
data

,patient_id,concept_id,value_numeric
0,16999416,1520176,0.6
1,16999416,1520177,93.0
2,16999416,1520177,94.0
3,16999416,1520177,94.0
4,16999416,1520177,93.0
...,...,...,...
7744120,19999784,1520694,32.2
7744121,19999784,1520694,32.4
7744122,19999784,1520694,32.4
7744123,19999784,1520694,32.9


### check the number of unique patients in the dataframe

In [40]:
print("Unique patients:",len(df["patient_id"].unique()))
#print("Unique diagnoses:",len(data["diagnosis_coded"].unique()))

Unique patients: 8297


## Getting value_numeric in different columns using get_dummies

In [41]:
df["concept_id"] = df["concept_id"].astype("category")
df['concept_id'].dtypes
dummies = pd.get_dummies(df['concept_id'])
dummies=dummies.replace(0,np.nan)
dummies=dummies.astype(float)
dummies.values[dummies == 1.0] = df['value_numeric']
dummies

,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,1520184,...,1529674,1529675,1529676,1529677,1529678,1529679,1529680,1529682,1529683,1529684
0,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,94.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,94.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7744120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7744121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7744122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7744123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Taking the patient_id column only to merge with dummies dataframe

In [42]:
data_id_only=df[['patient_id']]

## Now merging these two dataframes

In [43]:
dataframe=pd.concat([data_id_only,dummies],axis=1)
dataframe=dataframe.dropna(axis=1,how='all')
dataframe

,patient_id,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,...,1529674,1529675,1529676,1529677,1529678,1529679,1529680,1529682,1529683,1529684
0,16999416,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16999416,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16999416,NaN,NaN,94.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16999416,NaN,NaN,94.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16999416,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7744120,19999784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7744121,19999784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7744122,19999784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7744123,19999784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
squeezed= dataframe.groupby('patient_id').agg('mean')
squeezed

,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,1520184,...,1529674,1529675,1529676,1529677,1529678,1529679,1529680,1529682,1529683,1529684
patient_id,,,,,,,,,,,,,,,,,,,,,
16999416,NaN,0.600000,93.750000,NaN,NaN,NaN,NaN,NaN,13.428571,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16999815,1.296590,NaN,87.125000,NaN,NaN,NaN,NaN,NaN,11.333333,NaN,...,106.0,233.200000,69.0,175.0,NaN,NaN,NaN,NaN,NaN,NaN
17000103,4.956930,3.850000,87.954545,NaN,NaN,NaN,NaN,NaN,12.518519,NaN,...,86.8,191.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17000205,NaN,3.400000,90.375000,NaN,NaN,NaN,NaN,NaN,12.875000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17001006,2.330205,3.045455,89.494737,57.75,NaN,NaN,NaN,NaN,13.112150,NaN,...,80.4,179.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19998350,NaN,2.100000,81.666667,NaN,32.0,NaN,NaN,NaN,13.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19998497,NaN,1.500000,91.851852,24.00,199.5,NaN,NaN,NaN,14.923077,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19999379,NaN,NaN,92.000000,NaN,NaN,NaN,NaN,NaN,13.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## (Ignore it for now because we already got the dataframe with 8297 rows) ... Now we need to move the values so that every features' values stay in one row instead of multiple rows for one single patient, for multiple values for same concept, the values could stay in multiple rows

In [16]:
# df=dataframe
# feature_columns=list(df.columns)[1:]
# unique_id=sorted(set(df['patient_id']))

In [17]:
# def prepend_geek(name):
#     list_sugar=[str(x) for x in name]
#     non_null=[]
#     for i,e in enumerate(list_sugar):
#         if(e == 'nan'):
#             continue
#         non_null.append(e)
    
#     remaining_length=len(name)-len(non_null)
#     tail=['nan']*remaining_length
#     total = non_null + tail
#     total = non_null
#     return total

In [18]:
# def format_table(pdf):
#     #current_id=list(pdf['id']) 
#     current_df=pdf[feature_columns]
#     total_list=[]
#     for i,e in enumerate(feature_columns):

#         total_list.append(prepend_geek(current_df[e]))
#     remaining_table=pd.DataFrame(total_list).T
#     remaining_table.columns=feature_columns
#     remaining_table['patient_id']= [pdf['patient_id'].iloc[0]]*len(remaining_table)

#     return(remaining_table)

In [ ]:
# pdf_list=[]
# for i,e in enumerate(unique_id):
#     #print(i)
#     pdf=df[df['patient_id']==e]
#     pdf_list.append(format_table(pdf))

## (Ignore it too) Saving the list as pickle file so that if the concat does not work, we can have all the info in the list as pkl

In [ ]:
# with open('pdf_list.pkl', 'wb') as f:
#     pickle.dump(pdf_list, f)

In [45]:
# final_pdf=pd.concat(pdf_list,axis=0,ignore_index=True)
# final_pdf

## (Ignore it) Saving the dataframe as new dataframe

In [21]:
# final_pdf.to_csv("Final_dataframe_all_data_reduced_rows.csv")

In [22]:
# final_data=pd.read_csv("Final_dataframe_all_data_reduced_rows.csv")
# #final_data=final_data.reset_index()
# final_data=final_data.drop(['Unnamed: 0'], axis=1)
# first_column = final_data.pop('patient_id')
# final_data.insert(0, 'patient_id', first_column)
# final_data

,patient_id,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,...,1529674,1529675,1529676,1529677,1529678,1529679,1529680,1529682,1529683,1529684
0,16999416,NaN,0.6,93.0,NaN,NaN,NaN,NaN,NaN,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16999416,NaN,NaN,94.0,NaN,NaN,NaN,NaN,NaN,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16999416,NaN,NaN,94.0,NaN,NaN,NaN,NaN,NaN,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16999416,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16999416,NaN,NaN,95.0,NaN,NaN,NaN,NaN,NaN,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545260,19999784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545261,19999784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545262,19999784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545263,19999784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
squeezed=squeezed.reset_index()
squeezed

,patient_id,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,...,1529674,1529675,1529676,1529677,1529678,1529679,1529680,1529682,1529683,1529684
0,16999416,NaN,0.600000,93.750000,NaN,NaN,NaN,NaN,NaN,13.428571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16999815,1.296590,NaN,87.125000,NaN,NaN,NaN,NaN,NaN,11.333333,...,106.0,233.200000,69.0,175.0,NaN,NaN,NaN,NaN,NaN,NaN
2,17000103,4.956930,3.850000,87.954545,NaN,NaN,NaN,NaN,NaN,12.518519,...,86.8,191.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17000205,NaN,3.400000,90.375000,NaN,NaN,NaN,NaN,NaN,12.875000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17001006,2.330205,3.045455,89.494737,57.75,NaN,NaN,NaN,NaN,13.112150,...,80.4,179.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8292,19998350,NaN,2.100000,81.666667,NaN,32.0,NaN,NaN,NaN,13.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8293,19998497,NaN,1.500000,91.851852,24.00,199.5,NaN,NaN,NaN,14.923077,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8294,19999379,NaN,NaN,92.000000,NaN,NaN,NaN,NaN,NaN,13.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8295,19999625,0.960741,0.100000,86.000000,65.00,NaN,NaN,NaN,NaN,12.181818,...,50.5,111.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df1=target_variable_df
df1=df1.replace(np.nan,0)
df1

,patient_id,1384841,1384870,1384886,1384995,1385083,1385109,1385121,1385187,1397715,1474651
0,19563570,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,19557987,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,19557524,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19556594,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19555886,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
9995,17013053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9996,17009907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9997,17008429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9998,17006542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [57]:
document_dict_1 = pd.Series(df1['1384841'].values,index = df1.patient_id).to_dict()
document_dict_2 = pd.Series(df1["1384870"].values,index = df1.patient_id).to_dict()
document_dict_3 = pd.Series(df1["1384886"].values,index = df1.patient_id).to_dict()
document_dict_4 = pd.Series(df1["1384995"].values,index = df1.patient_id).to_dict()
document_dict_5 = pd.Series(df1["1385083"].values,index = df1.patient_id).to_dict()
document_dict_6 = pd.Series(df1["1385109"].values,index = df1.patient_id).to_dict()
document_dict_7 = pd.Series(df1["1385121"].values,index = df1.patient_id).to_dict()
document_dict_8 = pd.Series(df1["1385187"].values,index = df1.patient_id).to_dict()
document_dict_9 = pd.Series(df1["1397715"].values,index = df1.patient_id).to_dict()
document_dict_10 = pd.Series(df1["1474651"].values,index = df1.patient_id).to_dict()

# merged = pd.merge(df, df1, on ='patient_id')
# merged=merged.drop_duplicates()
# merged=merged.reset_index(drop=True)
# merged

In [58]:
document_dict_7

{19563570: 0.0,
 19557987: 0.0,
 19557524: 0.0,
 19556594: 0.0,
 19555886: 0.0,
 19552039: 0.0,
 19550378: 0.0,
 19547030: 0.0,
 19544131: 0.0,
 19541392: 1.0,
 19540374: 0.0,
 19539331: 0.0,
 19533644: 1.0,
 19532405: 0.0,
 19530697: 0.0,
 19530582: 0.0,
 19527441: 0.0,
 19525746: 0.0,
 19523386: 0.0,
 19523301: 1.0,
 19514027: 0.0,
 19512875: 1.0,
 19511048: 0.0,
 19510856: 0.0,
 19509250: 0.0,
 19508954: 0.0,
 19508874: 0.0,
 19503056: 0.0,
 19502544: 1.0,
 19501510: 1.0,
 19499595: 0.0,
 19493050: 0.0,
 19491367: 0.0,
 19491293: 0.0,
 19489981: 1.0,
 19489699: 0.0,
 19484821: 0.0,
 19480985: 0.0,
 19478816: 0.0,
 19478177: 0.0,
 19476335: 0.0,
 19475604: 1.0,
 19474929: 0.0,
 19472734: 0.0,
 19472091: 0.0,
 19470718: 0.0,
 19462678: 0.0,
 19462352: 0.0,
 19461413: 0.0,
 19459798: 0.0,
 19458578: 0.0,
 19454978: 1.0,
 19453139: 0.0,
 19452496: 0.0,
 19444227: 1.0,
 19442637: 0.0,
 19438923: 0.0,
 19438264: 0.0,
 19437332: 0.0,
 19427944: 1.0,
 19427173: 0.0,
 19423891: 0.0,
 1942290

In [59]:
squeezed['diagnosis_1'] = squeezed['patient_id'].map(document_dict_1) 
squeezed['diagnosis_2'] = squeezed['patient_id'].map(document_dict_2) 
squeezed['diagnosis_3'] = squeezed['patient_id'].map(document_dict_3) 
squeezed['diagnosis_4'] = squeezed['patient_id'].map(document_dict_4) 
squeezed['diagnosis_5'] = squeezed['patient_id'].map(document_dict_5) 
squeezed['diagnosis_6'] = squeezed['patient_id'].map(document_dict_6) 
squeezed['diagnosis_7'] = squeezed['patient_id'].map(document_dict_7) 
squeezed['diagnosis_8'] = squeezed['patient_id'].map(document_dict_8) 
squeezed['diagnosis_9'] = squeezed['patient_id'].map(document_dict_9) 
squeezed['diagnosis_10'] = squeezed['patient_id'].map(document_dict_10) 
squeezed

,patient_id,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,...,diagnosis_1,diagnosis_2,diagnosis_3,diagnosis_4,diagnosis_5,diagnosis_6,diagnosis_7,diagnosis_8,diagnosis_9,diagnosis_10
0,16999416,NaN,0.600000,93.750000,NaN,NaN,NaN,NaN,NaN,13.428571,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16999815,1.296590,NaN,87.125000,NaN,NaN,NaN,NaN,NaN,11.333333,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17000103,4.956930,3.850000,87.954545,NaN,NaN,NaN,NaN,NaN,12.518519,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,17000205,NaN,3.400000,90.375000,NaN,NaN,NaN,NaN,NaN,12.875000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,17001006,2.330205,3.045455,89.494737,57.75,NaN,NaN,NaN,NaN,13.112150,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8292,19998350,NaN,2.100000,81.666667,NaN,32.0,NaN,NaN,NaN,13.333333,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8293,19998497,NaN,1.500000,91.851852,24.00,199.5,NaN,NaN,NaN,14.923077,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8294,19999379,NaN,NaN,92.000000,NaN,NaN,NaN,NaN,NaN,13.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8295,19999625,0.960741,0.100000,86.000000,65.00,NaN,NaN,NaN,NaN,12.181818,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [60]:
dataset_for_modeling=squeezed

In [106]:
dataset_for_modeling

,patient_id,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,...,diagnosis_1,diagnosis_2,diagnosis_3,diagnosis_4,diagnosis_5,diagnosis_6,diagnosis_7,diagnosis_8,diagnosis_9,diagnosis_10
0,16999416,nan,0.6,93.75,nan,nan,nan,nan,nan,13.428571428571429,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16999815,1.29659,nan,87.125,nan,nan,nan,nan,nan,11.333333333333334,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17000103,4.95693,3.85,87.95454545454545,nan,nan,nan,nan,nan,12.518518518518519,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,17000205,nan,3.4,90.375,nan,nan,nan,nan,nan,12.875,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,17001006,2.3302050000000003,3.0454545454545454,89.49473684210527,57.75,nan,nan,nan,nan,13.11214953271028,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8292,19998350,nan,2.1,81.66666666666667,nan,32.0,nan,nan,nan,13.333333333333334,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8293,19998497,nan,1.5,91.85185185185185,24.0,199.5,nan,nan,nan,14.923076923076923,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8294,19999379,nan,nan,92.0,nan,nan,nan,nan,nan,13.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8295,19999625,0.960741,0.1,86.0,65.0,nan,nan,nan,nan,12.181818181818182,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
dataset_for_modeling.to_csv("dataset_for_modeling.csv")

## Trying Random forest for this dataset

In [ ]:
%%time
#dataset_for_modeling=dataset_for_modeling.applymap(str)

list_patients=dataset_for_modeling["patient_id"].unique()



for j in range(10):
    print("Diagnosis:",j+1)
    jj=10-j
    #print(jj)
    list_score=[]
    for i in range(len(list_patients)):
        print(i)
#         if i>5: #doing for only one patient for now
#             break
        patient_id=list_patients[i]

        #making test_set
        test_set=dataset_for_modeling.query("patient_id == @patient_id") #Taking only one patients all info as test set
        #test_set=test_set.replace("NaN",np.nan)
        test_set=test_set.dropna(axis=1, how='all')

        #making the training set
        list_columns=test_set.columns.tolist()
        data1=dataset_for_modeling[list_columns]
        training_set=data1.query("patient_id != @patient_id")
        training_set=training_set.reset_index()
        training_set=training_set.drop(['index'], axis=1)

        #print("test train set is done")
        #print(test_set)
        #print(training_set)
        col_n=len(test_set.columns)

        X_train=training_set.iloc[:,1:col_n-10]
        X_test=test_set.iloc[:,1:col_n-10]
        y_train=training_set.iloc[:,col_n-jj]
        y_test=test_set.iloc[:,col_n-jj]

        X_train=X_train.values
        X_test=X_test.values
        y_train=y_train.values
        y_test=y_test.values  

        #print("values of test train set are extracted")

        X_train[X_train == 'nan'] = 0.0
        X_test[X_test == 'nan'] = 0.0

        oversample = SMOTE()
        X_train_res, y_train_res = oversample.fit_resample(X_train, y_train)

        # Feature Scaling 
        sc_X = StandardScaler()
        X_train = sc_X.fit_transform(X_train_res)
        X_test = sc_X.transform(X_test)

        #print("defining rf")

        Rfclassifier = RandomForestClassifier(n_estimators=10, criterion='entropy', class_weight="balanced_subsample", random_state=42)

        Rfclassifier.fit(X_train_res, y_train_res.ravel())

        # Predicting the Test set results
        y_pred_rf = Rfclassifier.predict(X_test)
        #print('y_preds_rf:', y_pred_rf)

        # Making the confusion Matrix
        cm_rf = confusion_matrix(y_test, y_pred_rf)
        #print('confusion matrix_rf:', cm_rf)

        # predict for the patient

        cm_rf = confusion_matrix(y_test, y_pred_rf)


        def rf_report():
            patient_pred_rf = Rfclassifier.predict(X_test)
            report = classification_report(y_test, y_pred_rf, output_dict=True)
            df_report = pd.DataFrame(report).transpose()

            return df_report

        rf_report()
        df_rf=rf_report()
        #display(df_rf)
        val_rf=df_rf['f1-score']['macro avg']
        #print("For","Random forest macro avg F1 score",val_rf)
        list_score.append(val_rf)
    print("Average macro avg F1 score for",dataset_for_modeling.iloc[:,col_n-jj].name,"is",np.mean(list_score))

## Trying LightGBM for this dataset

In [ ]:
%%time
#dataset_for_modeling=dataset_for_modeling.applymap(str)

list_patients=dataset_for_modeling["patient_id"].unique()



for j in range(7,10):
    print("Diagnosis:",j+1)
    jj=10-j
    #print(jj)
    list_score=[]
    for i in range(len(list_patients)):
        print(i)
#         if i>2: #doing for only one patient for now
#             break
        patient_id=list_patients[i]

        #making test_set
        test_set=dataset_for_modeling.query("patient_id == @patient_id") #Taking only one patients all info as test set
        #test_set=test_set.replace("NaN",np.nan)
        test_set=test_set.dropna(axis=1, how='all')

        #making the training set
        list_columns=test_set.columns.tolist()
        data1=dataset_for_modeling[list_columns]
        training_set=data1.query("patient_id != @patient_id")
        training_set=training_set.reset_index()
        training_set=training_set.drop(['index'], axis=1)

        #print("test train set is done")
        #print(test_set)
        #print(training_set)
        col_n=len(test_set.columns)

        X_train=training_set.iloc[:,1:col_n-10]
        X_test=test_set.iloc[:,1:col_n-10]
        y_train=training_set.iloc[:,col_n-jj]
        y_test=test_set.iloc[:,col_n-jj]

        X_train=X_train.values
        X_test=X_test.values
        y_train=y_train.values
        y_test=y_test.values  

        #print("values of test train set are extracted")

        X_train[X_train == 'nan'] = 0.0
        X_test[X_test == 'nan'] = 0.0

        oversample = SMOTE()
        X_train_res, y_train_res = oversample.fit_resample(X_train, y_train)

        # Feature Scaling 
        sc_X = StandardScaler()
        X_train = sc_X.fit_transform(X_train_res)
        X_test = sc_X.transform(X_test)

        #print("defining rf")

        lgb_classifier = lgb.LGBMClassifier()
        lgb_classifier.fit(X_train_res, y_train_res)

        # Predicting the Test set results
        y_pred_rf = lgb_classifier.predict(X_test)
        #print('y_preds_rf:', y_pred_rf)

        # Making the confusion Matrix
        cm_rf = confusion_matrix(y_test, y_pred_rf)
        #print('confusion matrix_rf:', cm_rf)

        # predict for the patient

        cm_rf = confusion_matrix(y_test, y_pred_rf)


        def rf_report():
            patient_pred_rf = lgb_classifier.predict(X_test)
            report = classification_report(y_test, y_pred_rf, output_dict=True)
            df_report = pd.DataFrame(report).transpose()

            return df_report

        rf_report()
        df_rf=rf_report()
        #display(df_rf)
        val_rf=df_rf['f1-score']['macro avg']
        #print("For","Random forest macro avg F1 score",val_rf)
        list_score.append(val_rf)
    print("Average macro avg F1 score for",dataset_for_modeling.iloc[:,col_n-jj].name,"is",np.mean(list_score))

## Trying XGBoost for this dataset

In [ ]:
%%time
#dataset_for_modeling=dataset_for_modeling.applymap(str)

list_patients=dataset_for_modeling["patient_id"].unique()



for j in range(2,10):
    print("Diagnosis:",j+1)
    jj=10-j
    #print(jj)
    list_score=[]
    for i in range(len(list_patients)):
        print(i)
#         if i>2: #doing for only one patient for now
#             break
        patient_id=list_patients[i]

        #making test_set
        test_set=dataset_for_modeling.query("patient_id == @patient_id") #Taking only one patients all info as test set
        #test_set=test_set.replace("NaN",np.nan)
        test_set=test_set.dropna(axis=1, how='all')

        #making the training set
        list_columns=test_set.columns.tolist()
        data1=dataset_for_modeling[list_columns]
        training_set=data1.query("patient_id != @patient_id")
        training_set=training_set.reset_index()
        training_set=training_set.drop(['index'], axis=1)

        #print("test train set is done")
        #print(test_set)
        #print(training_set)
        col_n=len(test_set.columns)

        X_train=training_set.iloc[:,1:col_n-10]
        X_test=test_set.iloc[:,1:col_n-10]
        y_train=training_set.iloc[:,col_n-jj]
        y_test=test_set.iloc[:,col_n-jj]

        X_train=X_train.values
        X_test=X_test.values
        y_train=y_train.values
        y_test=y_test.values  

        #print("values of test train set are extracted")

        X_train[X_train == 'nan'] = 0.0
        X_test[X_test == 'nan'] = 0.0

        oversample = SMOTE()
        X_train_res, y_train_res = oversample.fit_resample(X_train, y_train)

        # Feature Scaling 
        sc_X = StandardScaler()
        X_train = sc_X.fit_transform(X_train_res)
        X_test = sc_X.transform(X_test)

        #print("defining rf")

        xgclassifier = XGBClassifier(n_estimators=10, criterion='entropy', class_weight="balanced_subsample", random_state=42)

        xgclassifier.fit(X_train_res, y_train_res.ravel())

        # Predicting the Test set results
        y_pred_rf = xgclassifier.predict(X_test)
        #print('y_preds_rf:', y_pred_rf)

        # Making the confusion Matrix
        cm_rf = confusion_matrix(y_test, y_pred_rf)
        #print('confusion matrix_rf:', cm_rf)

        # predict for the patient

        cm_rf = confusion_matrix(y_test, y_pred_rf)


        def rf_report():
            patient_pred_rf = xgclassifier.predict(X_test)
            report = classification_report(y_test, y_pred_rf, output_dict=True)
            df_report = pd.DataFrame(report).transpose()

            return df_report

        rf_report()
        df_rf=rf_report()
        #display(df_rf)
        val_rf=df_rf['f1-score']['macro avg']
        #print("For","Random forest macro avg F1 score",val_rf)
        list_score.append(val_rf)
    print("Average macro avg F1 score for",dataset_for_modeling.iloc[:,col_n-jj].name,"is",np.mean(list_score))

## Dataframe making according to professor's suggestion

In [145]:
%%time
dataset_for_modeling=pd.read_csv("dataset_for_modeling.csv")
dataset_for_modeling=dataset_for_modeling.drop(["Unnamed: 0"],axis=1)
dataset_for_modeling=dataset_for_modeling.drop(["diagnosis_1","diagnosis_2","diagnosis_3","diagnosis_4","diagnosis_5","diagnosis_6","diagnosis_7","diagnosis_8","diagnosis_9","diagnosis_10"],axis=1)
dataset_for_modeling=dataset_for_modeling.applymap(str)
dataset_for_modeling

CPU times: user 905 ms, sys: 35.4 ms, total: 940 ms
Wall time: 938 ms


,patient_id,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,...,1529674,1529675,1529676,1529677,1529678,1529679,1529680,1529682,1529683,1529684
0,16999416,nan,0.6,93.75,nan,nan,nan,nan,nan,13.428571428571429,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,16999815,1.29659,nan,87.125,nan,nan,nan,nan,nan,11.333333333333334,...,106.0,233.2,69.0,175.0,nan,nan,nan,nan,nan,nan
2,17000103,4.95693,3.85,87.95454545454545,nan,nan,nan,nan,nan,12.51851851851852,...,86.8,191.0,nan,nan,nan,nan,nan,nan,nan,nan
3,17000205,nan,3.4,90.375,nan,nan,nan,nan,nan,12.875,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,17001006,2.3302050000000003,3.0454545454545454,89.49473684210527,57.75,nan,nan,nan,nan,13.11214953271028,...,80.4,179.66666666666666,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8292,19998350,nan,2.1,81.66666666666667,nan,32.0,nan,nan,nan,13.333333333333334,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8293,19998497,nan,1.5,91.85185185185183,24.0,199.5,nan,nan,nan,14.923076923076923,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8294,19999379,nan,nan,92.0,nan,nan,nan,nan,nan,13.0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8295,19999625,0.960741,0.1,86.0,65.0,nan,nan,nan,nan,12.181818181818182,...,50.5,111.1,nan,nan,nan,nan,nan,nan,nan,nan


In [143]:
df1 = pd.read_csv('10k-diagnosis.csv',header=None)
df1 = df1.rename(columns={0: 'patient_id', 1: 'diagnosis_coded'})
df1.head()


list_index=[]
for i in range(df1.shape[0]): #Taking only the 10 diagnoses
    if df1.iloc[i,1]=='1385187' or df1.iloc[i,1]=='1385109' or df1.iloc[i,1]=='1384870' or df1.iloc[i,1]=='1384841' or df1.iloc[i,1]=='1384995' or df1.iloc[i,1]=='1397715' or df1.iloc[i,1]=='1384886' or df1.iloc[i,1]=='1474651' or df1.iloc[i,1]=='1385083' or df1.iloc[i,1]=='1385121':
        #print(i)
        list_index.append(i)
        
df1=df1.iloc[list_index]
df1=df1.reset_index()
df1=df1.drop(['index'], axis=1)
df1=df1.drop_duplicates()
df1=df1.reset_index()
df1=df1.drop(['index'], axis=1)
df1

,patient_id,diagnosis_coded
0,19563570,1384841
1,19557987,1384841
2,19557524,1384841
3,19556594,1384841
4,19555886,1384841
...,...,...
12577,17008429,1474651
12578,17008031,1474651
12579,17006542,1474651
12580,17002218,1474651


In [ ]:
uniquep=list(df1["patient_id"].unique())
allp=list(df1["patient_id"].values)
for i in range(len(uniquep)):
    for j in range(len(allp)):
        if uniquep[i]==allp[j]:
            allp.remove(allp[j])
            break

In [ ]:
dataset_feb04=dataset_for_modeling
for i in range(len(allp)):
#     if i>100:
#         break
    patient_id=allp[i]
    #print(patient_id)
    #print(i)
    for j in range(dataset_for_modeling.shape[0]):
#         if j>5:
#             break
        #print(j)
        patient_id_1=dataset_for_modeling["patient_id"][j]
        #print(patient_id_1)
        if str(patient_id)==str(patient_id_1):
            #print(j)
            patient_id_1=str(patient_id_1)
            df_2=dataset_for_modeling.query("patient_id == @patient_id_1")
            #display(df_2)
            dataset_feb04=pd.concat([dataset_feb04,df_2],axis=0)

In [211]:
dataset_feb04

,patient_id,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,...,1529674,1529675,1529676,1529677,1529678,1529679,1529680,1529682,1529683,1529684
0,16999416,nan,0.6,93.75,nan,nan,nan,nan,nan,13.428571428571429,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,16999815,1.29659,nan,87.125,nan,nan,nan,nan,nan,11.333333333333334,...,106.0,233.2,69.0,175.0,nan,nan,nan,nan,nan,nan
2,17000103,4.95693,3.85,87.95454545454545,nan,nan,nan,nan,nan,12.51851851851852,...,86.8,191.0,nan,nan,nan,nan,nan,nan,nan,nan
3,17000205,nan,3.4,90.375,nan,nan,nan,nan,nan,12.875,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,17001006,2.3302050000000003,3.0454545454545454,89.49473684210527,57.75,nan,nan,nan,nan,13.11214953271028,...,80.4,179.66666666666666,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,17035507,nan,2.0,94.0,nan,nan,nan,nan,nan,16.0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
82,17029062,nan,2.2,87.0,nan,nan,nan,nan,nan,13.0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
79,17027866,0.53941,2.0454545454545454,66.29787234042553,42.0,120.5,nan,nan,nan,13.916666666666666,...,111.3,251.0,nan,nan,nan,nan,nan,nan,nan,nan
41,17011846,1.3294186666666663,2.611764705882353,88.0,nan,167.0,nan,nan,nan,13.944444444444445,...,70.97777777777777,157.32142857142858,68.0,172.5,64.0,nan,nan,nan,nan,nan


In [ ]:
dataset_feb04=dataset_feb04.reset_index()
dataset_feb04=dataset_feb04.drop(["index"],axis=1)
dataset_feb04

In [ ]:
dataset_feb04["diagnosis_coded"]=np.nan
for i in range(df1.shape[0]):
    #print(i)
    for j in range(dataset_feb04.shape[0]):
        if str(df1["patient_id"][i])==str(dataset_feb04["patient_id"][j]):
            if str(dataset_feb04["diagnosis_coded"][j])=='nan':
                dataset_feb04["diagnosis_coded"][j]=df1["diagnosis_coded"][i]
                break
                
dataset_feb04

In [282]:
dataset_feb04=dataset_feb04.replace({'1385187':0,'1385109':1,'1384870':2,'1384841':3,'1384995':4,'1397715':5,'1384886':6,'1474651':7,'1385083':8,'1385121':9})
dataset_feb04

,patient_id,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,...,1529675,1529676,1529677,1529678,1529679,1529680,1529682,1529683,1529684,diagnosis_coded
0,16999416,nan,0.6,93.75,nan,nan,nan,nan,nan,13.428571428571429,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,6
1,16999815,1.29659,nan,87.125,nan,nan,nan,nan,nan,11.333333333333334,...,233.2,69.0,175.0,nan,nan,nan,nan,nan,nan,2
2,17000103,4.95693,3.85,87.95454545454545,nan,nan,nan,nan,nan,12.51851851851852,...,191.0,nan,nan,nan,nan,nan,nan,nan,nan,9
3,17000205,nan,3.4,90.375,nan,nan,nan,nan,nan,12.875,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,0
4,17001006,2.3302050000000003,3.0454545454545454,89.49473684210527,57.75,nan,nan,nan,nan,13.11214953271028,...,179.66666666666666,nan,nan,nan,nan,nan,nan,nan,nan,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10782,17035507,nan,2.0,94.0,nan,nan,nan,nan,nan,16.0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,7
10783,17029062,nan,2.2,87.0,nan,nan,nan,nan,nan,13.0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,7
10784,17027866,0.53941,2.0454545454545454,66.29787234042553,42.0,120.5,nan,nan,nan,13.916666666666666,...,251.0,nan,nan,nan,nan,nan,nan,nan,nan,7
10785,17011846,1.3294186666666663,2.611764705882353,88.0,nan,167.0,nan,nan,nan,13.944444444444445,...,157.32142857142858,68.0,172.5,64.0,nan,nan,nan,nan,nan,7


In [295]:
dataset_feb04.to_csv("dataset_feb04.csv")

## Finding number of patients who have multiple diagnosis

In [317]:
list_unique_patients=list(dataset_feb04["patient_id"].unique())
list_all_patients=list(dataset_feb04["patient_id"].values)
i=0
while len(list_unique_patients) > 0:
    patient_unique_number=len(list_unique_patients)
    print(patient_unique_number,"patients have at least",i+1,"diagnosis")
    uniquep=list_unique_patients
    allp=list_all_patients
    for k in range(len(uniquep)):
        for j in range(len(allp)):
            if uniquep[k]==allp[j]:
                allp.remove(allp[j])
                break
    list_unique_patients=list(set(allp))
    list_all_patients=allp
    i=i+1        

8297 patients have at least 1 diagnosis
1893 patients have at least 2 diagnosis
474 patients have at least 3 diagnosis
98 patients have at least 4 diagnosis
19 patients have at least 5 diagnosis
4 patients have at least 6 diagnosis
2 patients have at least 7 diagnosis


## Trying Random Forest for this dataset

In [ ]:
%%time
#dataset_feb04=dataset_feb04.applymap(str)

list_patients=dataset_feb04["patient_id"].unique()



# for j in range(10):
#     print("Diagnosis:",j+1)
#     jj=10-j
#     #print(jj)
#list_score=[]
for i in range(len(list_patients)):
    print(i)
#     if i>20: #doing for only one patient for now
#         break
    patient_id=list_patients[i]
    #patient_id=str(18230098)
    #making test_set
    test_set=dataset_feb04.query("patient_id == @patient_id") #Taking only one patients all info as test set
    #test_set=test_set.replace("NaN",np.nan)
    test_set=test_set.dropna(axis=1, how='all')

    #making the training set
    list_columns=test_set.columns.tolist()
    data1=dataset_feb04[list_columns]
    training_set=data1.query("patient_id != @patient_id")
    training_set=training_set.reset_index()
    training_set=training_set.drop(['index'], axis=1)

    #print("test train set is done")
    #print(test_set)
    #print(training_set)
    col_n=len(test_set.columns)
    
    #display(training_set)
    #display(test_set)

    X_train=training_set.iloc[:,1:col_n-1]
    X_test=test_set.iloc[:,1:col_n-1]
    y_train=training_set.iloc[:,col_n-1]
    y_test=test_set.iloc[:,col_n-1]

#     display(X_train)
#     display(X_test)
#     display(y_train)
#     display(y_test)

    X_train=X_train.values
    X_test=X_test.values
    y_train=y_train.values
    y_test=y_test.values  

    #print("values of test train set are extracted")

    X_train[X_train == 'nan'] = 0.0
    X_test[X_test == 'nan'] = 0.0

    oversample = SMOTE()
    X_train_res, y_train_res = oversample.fit_resample(X_train, y_train)

    # Feature Scaling 
    sc_X = StandardScaler()
    X_train = sc_X.fit_transform(X_train_res)
    X_test = sc_X.transform(X_test)

    #print("defining rf")

    Rfclassifier = RandomForestClassifier(n_estimators=10, criterion='entropy', class_weight="balanced_subsample", random_state=42)

    Rfclassifier.fit(X_train_res, y_train_res.ravel())

    # Predicting the Test set results
    y_pred_rf = Rfclassifier.predict(X_test)
    #print('y_preds_rf:', y_pred_rf)

    # Making the confusion Matrix
    cm_rf = confusion_matrix(y_test, y_pred_rf)
    #print('confusion matrix_rf:', cm_rf)

    # predict for the patient

    cm_rf = confusion_matrix(y_test, y_pred_rf)


    def rf_report():
        patient_pred_rf = Rfclassifier.predict(X_test)
        report = classification_report(y_test, y_pred_rf, output_dict=True)
        df_report = pd.DataFrame(report).transpose()

        return df_report

    rf_report()
    df_rf=rf_report()
    #display(df_rf)
    val_rf=df_rf['f1-score']['macro avg']
    #print("For","Random forest macro avg F1 score",val_rf)
    list_score.append(val_rf)
print("Average macro avg F1 score is",np.mean(list_score))